<a href="https://colab.research.google.com/github/Nietzsche2000/TransformerExpNLP/blob/main/Transformers4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

2024-01-18 11:39:17.712220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-18 11:39:17.712291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-18 11:39:17.714327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-18 11:39:17.725126: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 11:39:19.258542: W tensorflow/compiler/tf2

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import spacy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Attention(nn.Module):
    def __init__(self, embed_size, heads):

        super(Attention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embed size needs to be divisible by heads"

        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)

        self.fc_out = nn.Linear(heads * self.head_dim, embed_size, bias=False)

    def forward(self, query, keys, values, mask):

        N = query.shape[0]
        query_len = query.shape[1]
        key_len = keys.shape[1]
        value_len = values.shape[1]

        queries = query.reshape(N, query_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        values = values.reshape(N, value_len, self.heads, self.head_dim)

        queries = self.queries(queries)
        keys = self.keys(keys)
        values = self.values(values)

        qk = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            qk = qk.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(qk / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out


In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, embed_size, heads, dropout):
        super(EncoderLayer, self).__init__()
        self.attention = Attention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size , embed_size), # can change 2
            nn.ReLU(),
            nn.Linear(embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask):
        attention = self.attention(query, key, value, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [ ]:
class Encoder(nn.Module):
    def __init__(
        self,
        vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        dropout,
        max_length
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                EncoderLayer(
                    embed_size,
                    heads,
                    dropout=dropout,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

        out = self.dropout(
            self.word_embedding(x) + self.position_embedding(positions)
        )

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, embed_size, heads, dropout, device):
        super(DecoderLayer, self).__init__()
        self.attention = Attention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.encoder_layer = EncoderLayer(
            embed_size, heads, dropout
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, key, value, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.encoder_layer(query, key, value, src_mask)
        return out

In [ ]:
class Decoder(nn.Module):
    def __init__(
        self,
        vocab_size,
        embed_size,
        num_layers,
        heads,
        dropout,
        device,
        max_length
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderLayer(
                    embed_size,
                    heads,
                    dropout,
                    device
                )
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)
        return out



In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx,
                 embed_size=512, num_layers=6,
                 heads=8, dropout=0, max_length=100):
        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            dropout,
            max_length
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            dropout,
            device,
            max_length
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [ ]:
import spacy
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from torchtext import data, datasets

In [ ]:
spacy_eng = spacy.load("en_core_web_sm")
spacy_ger = spacy.load("de_core_news_sm")

def tokenize_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]

def tokenize_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]

class Vocabulary:
    def __init__(self, tokenizer):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.tokenizer = tokenizer

    def build_vocabulary(self, sentence_list):
        idx = 4
        for sentence in sentence_list:
            for word in self.tokenizer(sentence):
                if word not in self.stoi:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, german, english, german_vocab, english_vocab):
        self.german = german
        self.english = english
        self.german_vocab = german_vocab
        self.english_vocab = english_vocab

    def __len__(self):
        return len(self.german)

    def __getitem__(self, index):
        german_sentence = self.german[index]
        english_sentence = self.english[index]

        german_numerical = [self.german_vocab.stoi["<SOS>"]]
        german_numerical += self.german_vocab.numericalize(german_sentence)
        german_numerical.append(self.german_vocab.stoi["<EOS>"])

        english_numerical = [self.english_vocab.stoi["<SOS>"]]
        english_numerical += self.english_vocab.numericalize(english_sentence)
        english_numerical.append(self.english_vocab.stoi["<EOS>"])

        return torch.tensor(german_numerical), torch.tensor(english_numerical)

In [ ]:
german_sentences = ["Ein Satz in Deutsch.", "Noch ein Satz."]
english_sentences = ["A sentence in English.", "Another sentence."]

german_vocab = Vocabulary(tokenize_ger)
english_vocab = Vocabulary(tokenize_eng)
german_vocab.build_vocabulary(german_sentences)
english_vocab.build_vocabulary(english_sentences)

dataset = CustomDataset(german_sentences, english_sentences, german_vocab, english_vocab)

pad_idx = english_vocab.stoi["<PAD>"]
batch_size = 2

def collate_fn(batch):
    german_batch, english_batch = [], []
    for german_item, english_item in batch:
        german_batch.append(torch.tensor(german_item))
        english_batch.append(torch.tensor(english_item))
    german_batch = pad_sequence(german_batch, batch_first=True, padding_value=german_vocab.stoi["<PAD>"])
    english_batch = pad_sequence(english_batch, batch_first=True, padding_value=english_vocab.stoi["<PAD>"])
    return german_batch, english_batch

loader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)

# Transformer model
src_vocab_size = len(german_vocab.stoi)
trg_vocab_size = len(english_vocab.stoi)
src_pad_idx = german_vocab.stoi["<PAD>"]
trg_pad_idx = english_vocab.stoi["<PAD>"]
model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx).to(device)

num_epochs = 100
learning_rate = 0.001

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch_idx, batch in tqdm(enumerate(loader), total=len(loader)):
        src = batch[0].to(device)
        trg = batch[1].to(device)

        output = model(src, trg[:, :-1])
        output = output.reshape(-1, output.shape[2])
        trg = trg[:, 1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch}/{num_epochs} Loss: {epoch_loss / len(loader)}")


# torch.save(model.state_dict(), "transformer.pth")
# model.load_state_dict(torch.load("transformer.pth"))

model.eval()
sentence = "Ein Satz in Deutsch."
tokens = [german_vocab.stoi["<SOS>"]] + german_vocab.numericalize(sentence) + [german_vocab.stoi["<EOS>"]]
tokens = torch.LongTensor(tokens).unsqueeze(0).to(device)

with torch.no_grad():
    outputs = model(tokens, tokens)
    outputs = torch.argmax(outputs.squeeze(0), dim=1).tolist()

translation = [english_vocab.itos[idx] for idx in outputs]
print(translation)
